In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import tensorflow as tf
import optuna
from optuna.samplers import TPESampler

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Input, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2

In [ ]:
TRAINING_DIR = "./cats-v-non-cats/training/"
VALIDATION_DIR = "./cats-v-non-cats/validation/"
TESTING_DIR = "./cats-v-non-cats/test/"

# Define whether to include test split or not
INCLUDE_TEST = True

In [ ]:
gpu_available = tf.config.list_physical_devices('GPU')

if gpu_available:
    print("GPU is available and being used.")
    print("GPU Details:", gpu_available)
else:
    print("GPU is not available. Check your Colab runtime settings.")

GPU is available and being used.
GPU Details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# load data

In [ ]:
# Unzip dataset in Google Colab

zip_file_path = "/content/cats-v-non-cats.zip"
extract_dir = "/content/cats-v-non-cats/"

os.makedirs(extract_dir, exist_ok=True)

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # Extract only the files we want, avoiding the extra directory structure and __MACOSX
            if not file_info.filename.startswith('__MACOSX/'):
                # Remove the leading 'cats-v-dogs/' from the filename if it exists
                arcname = file_info.filename
                if arcname.startswith('cats-v-non-cats/'):
                    arcname = arcname[len('cats-v-non-cats/'):]

                # Only extract if the filename is not empty after removing the prefix
                if arcname:
                    file_info.filename = arcname
                    zip_ref.extract(file_info, extract_dir)


    print(f"File extracted to: {extract_dir}")

    # Remove the __MACOSX folder if it exists
    macosx_folder = os.path.join("/content/", '__MACOSX')
    if os.path.exists(macosx_folder):
        shutil.rmtree(macosx_folder)
        print(f"Folder '{macosx_folder}' deleted successfully.")

except Exception as e:
    print(f"An error occurred during unzipping: {e}")

In [ ]:
# Set up data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

validation_gen = ImageDataGenerator(rescale=1./255)

if INCLUDE_TEST:
    test_gen = ImageDataGenerator(rescale=1./255)

# model

In [ ]:
def create_tuned_model(reg_strength=0.0001, dropout_conv=0.2, dropout_dense=0.4,
                      dense_units=512, filters_multiplier=1):

    inputs = Input(shape=(128, 128, 3))

    x = MaxPooling2D(2, 2)(inputs)

    filters1 = int(32 * filters_multiplier)
    filters2 = int(64 * filters_multiplier)
    filters3 = int(128 * filters_multiplier)

    # First block
    x = Conv2D(filters1, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters1, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = MaxPooling2D(2, 2)(x)
    x = Dropout(dropout_conv)(x)

    # Second block
    x = Conv2D(filters2, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters2, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = MaxPooling2D(2, 2)(x)
    x = Dropout(dropout_conv)(x)

    # Third block
    x = Conv2D(filters3, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters3, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = MaxPooling2D(2, 2)(x)
    x = Dropout(dropout_conv)(x)

    # Global pooling and dense layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(dense_units, activation='relu', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_dense)(x)
    x = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_strength))(x)

    return Model(inputs=inputs, outputs=x)

In [ ]:
def train_and_evaluate_model(config, epochs=5, param_name=None, param_value=None):
    try:
        model = create_tuned_model(
            reg_strength=config['reg_strength'],
            dropout_conv=config['dropout_conv'],
            dropout_dense=config['dropout_dense'],
            dense_units=config['dense_units'],
            filters_multiplier=config['filters_multiplier']
        )

        optimizer = Adam(
            learning_rate=config['learning_rate'],
            beta_1=config['beta_1'],
            beta_2=config['beta_2']
        )

        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', 'AUC']
        )

        train_generator = train_gen.flow_from_directory(
            TRAINING_DIR,
            target_size=(128, 128),
            batch_size=config['batch_size'],
            class_mode='binary',
            shuffle=True
        )

        validation_generator = validation_gen.flow_from_directory(
            VALIDATION_DIR,
            target_size=(128, 128),
            batch_size=config['batch_size'],
            class_mode='binary',
            shuffle=False
        )

        if INCLUDE_TEST:
            test_generator = test_gen.flow_from_directory(
                TESTING_DIR,
                target_size=(128, 128),
                batch_size=config['batch_size'],
                class_mode='binary',
                shuffle=True
            )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.2,
            patience=5,
            min_lr=1e-8,
            verbose=1
        )

        early_stop = EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            verbose=1,
            mode='max'
        )

        callbacks = [reduce_lr, early_stop]

        history = model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=epochs,
            verbose=1,
            callbacks=[reduce_lr, early_stop]
        )

        val_loss, val_accuracy, val_auc = model.evaluate(validation_generator, verbose=0)

        # Get test metrics if test set is available
        test_results = {}
        if INCLUDE_TEST:
            test_loss, test_accuracy, test_auc = model.evaluate(test_generator, verbose=0)
            test_results = {
                'test_accuracy': test_accuracy,
                'test_auc': test_auc,
                'test_loss': test_loss
            }

        results = {
            'val_accuracy': val_accuracy,
            'val_auc': val_auc,
            'val_loss': val_loss,
            'history': history.history,
            'batch_size_used': config['batch_size']
        }

        # Add test results if available
        results.update(test_results)

        return results

    except Exception as e:
        print(f"Error in training: {e}")
        return {
            'val_accuracy': 0.0,
            'val_auc': 0.0,
            'val_loss': float('inf'),
            'error': str(e)
        }

In [ ]:
def objective(trial):
    config = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'reg_strength': trial.suggest_float('reg_strength', 1e-6, 1e-2, log=True),
        'dropout_conv': trial.suggest_float('dropout_conv', 0.1, 0.4),
        'dropout_dense': trial.suggest_float('dropout_dense', 0.2, 0.7),
        'dense_units': trial.suggest_categorical('dense_units', [128, 256, 512, 1024, 2048]),
        'filters_multiplier': trial.suggest_float('filters_multiplier', 0.5, 2.0),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        'beta_1': trial.suggest_float('beta_1', 0.7, 0.99),
        'beta_2': trial.suggest_float('beta_2', 0.9, 0.9999)
    }

    try:
        result = train_and_evaluate_model(
            config,
            epochs=10,
            param_name=f"trial_{trial.number}",
            param_value="bayesian"
        )
        accuracy = result['val_accuracy']

        trial.set_user_attr('val_auc', result['val_auc'])
        trial.set_user_attr('val_loss', result['val_loss'])

        print(f"Trial {trial.number}: Accuracy = {accuracy:.4f}")

        return accuracy

    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return 0.0

def run_bayesian_optimization(n_trials=50, timeout=None):

    print(f"Starting Bayesian Optimization with {n_trials} trials...")

    study = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=42),
        study_name='cnn_hyperparameter_optimization'
    )

    # Run optimization
    study.optimize(objective, n_trials=n_trials, timeout=timeout)

    print(f"\n{'='*60}")
    print("BAYESIAN OPTIMIZATION RESULTS")
    print(f"{'='*60}")

    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Best trial number: {study.best_trial.number}")
    print(f"Best validation accuracy: {study.best_value:.4f}")

    print(f"\n🏆 Best hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")

    # Get additional metrics for best trial
    best_trial = study.best_trial
    if 'val_auc' in best_trial.user_attrs:
        print(f"\nBest trial metrics:")
        print(f"  Validation AUC: {best_trial.user_attrs['val_auc']:.4f}")
        print(f"  Validation Loss: {best_trial.user_attrs['val_loss']:.4f}")

    return study

In [ ]:
study = run_bayesian_optimization(n_trials=50)  # Increase to 100-200 for production

[I 2025-11-14 22:23:22,547] A new study created in memory with name: cnn_hyperparameter_optimization


Starting Bayesian Optimization with 20 trials...
Found 8544 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Epoch 1/15
267/267 ━━━━━━━━━━━━━━━━━━━━ 80s 255ms/step - AUC: 0.7549 - accuracy: 0.6884 - loss: 5.5385 - val_AUC: 0.6308 - val_accuracy: 0.5000 - val_loss: 3.7692 - learning_rate: 3.1489e-04
Epoch 2/15
267/267 ━━━━━━━━━━━━━━━━━━━━ 65s 242ms/step - AUC: 0.8550 - accuracy: 0.7811 - loss: 2.4943 - val_AUC: 0.8889 - val_accuracy: 0.7341 - val_loss: 1.4339 - learning_rate: 3.1489e-04
Epoch 3/15
267/267 ━━━━━━━━━━━━━━━━━━━━ 66s 247ms/step - AUC: 0.8875 - accuracy: 0.8116 - loss: 1.1581 - val_AUC: 0.8855 - val_accuracy: 0.8099 - val_loss: 0.8616 - learning_rate: 3.1489e-04
Epoch 4/15
267/267 ━━━━━━━━━━━━━━━━━━━━ 64s 241ms/step - AUC: 0.9056 - accuracy: 0.8259 - loss: 0.7702 - val_AUC: 0.9035 - val_accuracy: 0.7041 - val_loss: 0.8725 - learning_rate: 3.1489e-04
Epoch 5/15
267/267 ━━━━━━━━━━━━━━━━━━━━ 65s 243ms/step - AUC:

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
[I 2025-11-14 22:39:54,529] Trial 0 finished with value: 0.8904494643211365 and parameters: {'learning_rate': 0.00031489116479568613, 'reg_strength': 0.006351221010640704, 'dropout_conv': 0.31959818254342154, 'dropout_dense': 0.49932924209851826, 'dense_units': 1024, 'filters_multiplier': 1.5621088666940683, 'batch_size': 32, 'beta_1': 0.7527292404900592, 'beta_2': 0.9183221105343581}. Best is trial 0 with value: 0.8904494643211365.


Trial 0: Accuracy = 0.8904
Found 8544 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Epoch 1/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 85s 552ms/step - AUC: 0.7874 - accuracy: 0.7080 - loss: 0.6722 - val_AUC: 0.7648 - val_accuracy: 0.5000 - val_loss: 0.9804 - learning_rate: 1.6480e-04
Epoch 2/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 65s 482ms/step - AUC: 0.8701 - accuracy: 0.7924 - loss: 0.5545 - val_AUC: 0.6375 - val_accuracy: 0.5000 - val_loss: 1.2761 - learning_rate: 1.6480e-04
Epoch 3/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 64s 478ms/step - AUC: 0.9000 - accuracy: 0.8243 - loss: 0.4960 - val_AUC: 0.6730 - val_accuracy: 0.5103 - val_loss: 1.5167 - learning_rate: 1.6480e-04
Epoch 4/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 64s 477ms/step - AUC: 0.9237 - accuracy: 0.8480 - loss: 0.4474 - val_AUC: 0.8669 - val_accuracy: 0.7322 - val_loss: 0.7117 - learning_rate: 1.6480e-04
Epoch 5/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 64s 477ms/step - AUC: 0.9290 - accuracy: 0.

[I 2025-11-14 22:56:31,933] Trial 1 finished with value: 0.8838951587677002 and parameters: {'learning_rate': 0.00016480446427978953, 'reg_strength': 0.0001256104370001356, 'dropout_conv': 0.22958350559263474, 'dropout_dense': 0.34561457009902097, 'dense_units': 128, 'filters_multiplier': 1.6777639420895203, 'batch_size': 64, 'beta_1': 0.8761880070514171, 'beta_2': 0.9170353599563604}. Best is trial 0 with value: 0.8904494643211365.


...

Trial 17: Accuracy = 0.8296
Found 8544 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Epoch 1/15
534/534 ━━━━━━━━━━━━━━━━━━━━ 89s 134ms/step - AUC: 0.7496 - accuracy: 0.6889 - loss: 0.9027 - val_AUC: 0.8389 - val_accuracy: 0.7537 - val_loss: 0.7847 - learning_rate: 3.5346e-05
Epoch 2/15
534/534 ━━━━━━━━━━━━━━━━━━━━ 66s 123ms/step - AUC: 0.8066 - accuracy: 0.7324 - loss: 0.8333 - val_AUC: 0.8281 - val_accuracy: 0.6124 - val_loss: 1.2433 - learning_rate: 3.5346e-05
Epoch 3/15
534/534 ━━━━━━━━━━━━━━━━━━━━ 65s 122ms/step - AUC: 0.8233 - accuracy: 0.7446 - loss: 0.8058 - val_AUC: 0.8170 - val_accuracy: 0.5506 - val_loss: 1.7100 - learning_rate: 3.5346e-05
Epoch 4/15
534/534 ━━━━━━━━━━━━━━━━━━━━ 66s 123ms/step - AUC: 0.8516 - accuracy: 0.7751 - loss: 0.7552 - val_AUC: 0.7999 - val_accuracy: 0.5009 - val_loss: 1.8283 - learning_rate: 3.5346e-05
Epoch 5/15
534/534 ━━━━━━━━━━━━━━━━━━━━ 65s 122ms/step - AUC: 0.8559 - accuracy: 0

[I 2025-11-15 03:47:55,162] Trial 18 finished with value: 0.7537453174591064 and parameters: {'learning_rate': 3.534620594229477e-05, 'reg_strength': 0.00040065791875913605, 'dropout_conv': 0.3530135638866639, 'dropout_dense': 0.43467313388179807, 'dense_units': 256, 'filters_multiplier': 1.4097117273279314, 'batch_size': 16, 'beta_1': 0.8239971780532256, 'beta_2': 0.9488904980114387}. Best is trial 15 with value: 0.9428839087486267.


Trial 18: Accuracy = 0.7537
Found 8544 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.
Epoch 1/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 120s 693ms/step - AUC: 0.7572 - accuracy: 0.6940 - loss: 0.8472 - val_AUC: 0.6594 - val_accuracy: 0.5000 - val_loss: 1.2212 - learning_rate: 4.1332e-04
Epoch 2/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 65s 486ms/step - AUC: 0.8642 - accuracy: 0.7917 - loss: 0.6818 - val_AUC: 0.7199 - val_accuracy: 0.5028 - val_loss: 1.1134 - learning_rate: 4.1332e-04
Epoch 3/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 65s 484ms/step - AUC: 0.8877 - accuracy: 0.8116 - loss: 0.6274 - val_AUC: 0.8545 - val_accuracy: 0.5421 - val_loss: 1.1984 - learning_rate: 4.1332e-04
Epoch 4/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 64s 479ms/step - AUC: 0.9183 - accuracy: 0.8447 - loss: 0.5498 - val_AUC: 0.9328 - val_accuracy: 0.7444 - val_loss: 0.6796 - learning_rate: 4.1332e-04
Epoch 5/15
134/134 ━━━━━━━━━━━━━━━━━━━━ 65s 483ms/step - AUC: 0.9368 - accuracy: 

[I 2025-11-15 04:05:13,671] Trial 19 finished with value: 0.9428839087486267 and parameters: {'learning_rate': 0.0004133206909441453, 'reg_strength': 0.0002524047272916008, 'dropout_conv': 0.30200401139633054, 'dropout_dense': 0.35783750523357405, 'dense_units': 256, 'filters_multiplier': 1.8062473253541729, 'batch_size': 64, 'beta_1': 0.8864019818009142, 'beta_2': 0.9679044493038355}. Best is trial 15 with value: 0.9428839087486267.


Trial 19: Accuracy = 0.9429

BAYESIAN OPTIMIZATION RESULTS
Number of finished trials: 20
Best trial number: 15
Best validation accuracy: 0.9429

🏆 Best hyperparameters:
  learning_rate: 0.0005039156628791341
  reg_strength: 0.00036346431666936047
  dropout_conv: 0.1441896605006296
  dropout_dense: 0.3976953474897748
  dense_units: 256
  filters_multiplier: 1.452552653056354
  batch_size: 64
  beta_1: 0.819282704275543
  beta_2: 0.9547761923247599

Best trial metrics:
  Validation AUC: 0.9811
  Validation Loss: 0.2666


In [ ]:
# Analyze and Visualize Results
def analyze_bayesian_results(study):
    print(f"\n{'='*60}")
    print("🔥 TRAINING FINAL MODEL WITH BEST PARAMETERS")
    print(f"{'='*60}")

    best_config = study.best_params

    # Train the final model with the best hyperparameters
    final_result = train_and_evaluate_model(
        best_config,
        epochs=60,  # More epochs for final training
        param_name="bayesian_best",
        param_value="final"
    )

    # Plot optimization history
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Optimization history
    trial_numbers = [trial.number for trial in study.trials]
    trial_values = [trial.value if trial.value is not None else 0 for trial in study.trials]

    axes[0, 0].plot(trial_numbers, trial_values, 'b-', alpha=0.6)
    axes[0, 0].scatter(trial_numbers, trial_values, c=trial_values, cmap='viridis', s=30)
    # Removed baseline plot line
    # axes[0, 0].axhline(y=baseline_result['val_accuracy'], color='red', linestyle='--',
    #                    label=f'Baseline ({baseline_result["val_accuracy"]:.4f})')
    axes[0, 0].set_xlabel('Trial Number')
    axes[0, 0].set_ylabel('Validation Accuracy')
    axes[0, 0].set_title('Optimization History')
    # axes[0, 0].legend() # Removed legend since baseline is removed
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Parameter importance (if available)
    try:
        importance = optuna.importance.get_param_importances(study)
        params = list(importance.keys())
        values = list(importance.values())

        axes[0, 1].barh(params, values)
        axes[0, 1].set_xlabel('Importance')
        axes[0, 1].set_title('Hyperparameter Importance')
        axes[0, 1].grid(True, alpha=0.3)
    except:
        axes[0, 1].text(0.5, 0.5, 'Parameter importance\nnot available\n(need more trials)',
                        ha='center', va='center', transform=axes[0, 1].transAxes)
        axes[0, 1].set_title('Hyperparameter Importance')


    # 3. Best vs worst trials comparison
    best_trials = sorted(study.trials, key=lambda t: t.value if t.value is not None else 0, reverse=True)[:5]
    worst_trials = sorted(study.trials, key=lambda t: t.value if t.value is not None else 0)[:5]


    best_values = [t.value for t in best_trials if t.value is not None]
    worst_values = [t.value for t in worst_trials if t.value is not None]

    axes[1, 0].bar(range(len(best_values)), best_values, color='green', alpha=0.7, label='Best 5 trials')
    axes[1, 0].bar(range(len(best_values), len(best_values) + len(worst_values)),
                   worst_values, color='red', alpha=0.7, label='Worst 5 trials')
    axes[1, 0].set_ylabel('Validation Accuracy')
    axes[1, 0].set_title('Best vs Worst Trials')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Learning rate vs accuracy scatter
    lr_values = []
    acc_values = []
    for trial in study.trials:
        if trial.value is not None and 'learning_rate' in trial.params:
            lr_values.append(trial.params['learning_rate'])
            acc_values.append(trial.value)

    if lr_values:
        axes[1, 1].scatter(lr_values, acc_values, alpha=0.6, c=acc_values, cmap='viridis')
        axes[1, 1].set_xscale('log')
        axes[1, 1].set_xlabel('Learning Rate')
        axes[1, 1].set_ylabel('Validation Accuracy')
        axes[1, 1].set_title('Learning Rate vs Accuracy')
        axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('bayesian_optimization_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Save the final trained model
    final_model = create_tuned_model(**best_config)
    optimizer = Adam(
        learning_rate=best_config['learning_rate'],
        beta_1=best_config['beta_1'],
        beta_2=best_config['beta_2']
    )
    final_model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC']
    )

    callbacks = [
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=1e-8, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True, verbose=1, mode='max')
    ]

    final_model.fit(
        train_gen,
        validation_data=validation_gen,
        epochs=60, # Use same number of epochs as in analysis
        verbose=1,
        callbacks=callbacks
    )

    model_filename = "best_bayesian_model.h5"
    final_model.save(model_filename)
    print(f"\n🏆 Best Bayesian model saved as: {model_filename}")
    final_result['model_filename'] = model_filename


    return final_result, best_config

# Run analysis
if 'study' in locals():
    bayesian_final_result, bayesian_best_config = analyze_bayesian_results(study)
    print("✅ Analysis complete!")
else:
    print("❌ Run Bayesian optimization first!")

In [ ]:
# Final Summary and Model Comparison
def final_summary():
    print(f"\n{'='*70}")
    print("🎯 BAYESIAN OPTIMIZATION SUMMARY")
    print(f"{'='*70}")

    if 'bayesian_final_result' in locals():
        optimized_acc = bayesian_final_result['val_accuracy']

        print(f"\n🏆 Best Configuration Found:")
        for param, value in bayesian_best_config.items():
            print(f"  {param:<18}: {value}")

        print(f"\n🔍 Key Insights:")
        print(f"  • Total trials run: {len(study.trials)}")
        print(f"  • Best trial: #{study.best_trial.number}")
        print(f"  • Search space explored efficiently using Bayesian optimization")
        print(f"  • Model saved as: {bayesian_final_result['model_filename']}")

        print(f"\n✅ Bayesian optimization process completed.")


    else:
        print("❌ Bayesian optimization results not available.")

    print(f"\n🚀 Next Steps:")
    print(f"  • Use the best model for production")
    print(f"  • Consider ensemble methods")
    print(f"  • Test on holdout data")
    print(f"  • Monitor performance in production")

# Run final summary
final_summary()